# Ontology creation for TCE/RN

In [1]:
import sys, os

from owlready2 import *

In [ ]:
from rdflib import Graph
g = Graph()
g.parse("DUL.owl", format="turtle")     # seu arquivo em Turtle
g.serialize("DUL.rdf", format="xml")    # salva em RDF/XML


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


<Graph identifier=N230f995ec3b54a63ac6c7f3ac3552b4a (<class 'rdflib.graph.Graph'>)>

In [2]:
onto_path.append(os.path.expanduser("~/ontologias"))

In [ ]:
world = World()

In [30]:
dul = world.get_ontology("file://" + os.path.abspath("DUL.rdf")).load()
tce = world.get_ontology("http://tce.rn.gov.br/tce.owl")

In [ ]:
# Ontologia do TCE
tce = world.get_ontology("http://tce.rn.gov.br/tce.owl")
with tce:
    # ======== CLASSES (alinhadas ao DUL) ========
    class Processo(dul.SocialObject): pass                     # Processo (dossiê/autos)
    class Acordao(dul.SocialObject): pass                      # Texto/ato decisório como objeto social
    class SessaoJulgamento(dul.Event): pass                    # Evento (perdurant)
    class Obrigacao(dul.SocialObject): pass                    # A entidade normativa "obrigação"
    class MultaCominatoria(dul.SocialObject): pass             # A multa por descumprimento (título)
    class OrgaoResponsavel(dul.Agent): pass                    # Órgão (agente coletivo)
    class PessoaResponsavel(dul.Agent): pass                   # Pessoa física (agente)

    # (Opcional) Vocabulário controlado para tipo da obrigação
    class TipoObrigacao(Thing):
        namespace = tce
        pass
    # Instâncias do vocabulário
    tipo_fazer       = TipoObrigacao("Fazer")
    tipo_nao_fazer   = TipoObrigacao("NaoFazer")

    # ======== PROPRIEDADES OBJETO ========
    class isDerivedFrom(ObjectProperty):
        # Obrigação deriva de um Acórdão
        domain = [Obrigacao]
        range  = [Acordao]

    class emittedIn(ObjectProperty):
        # Acordão emitido em uma Sessão de Julgamento
        domain = [Acordao]
        range  = [SessaoJulgamento]

    class temOrgaoResponsavel(ObjectProperty, FunctionalProperty):
        # Campo: orgao_responsavel (jurídica). Se não houver, "Desconhecido".
        domain = [Obrigacao]
        range  = [OrgaoResponsavel]

    class temPessoaResponsavel(ObjectProperty):
        # Pessoas responsáveis (podem ser várias)
        domain = [Obrigacao]
        range  = [PessoaResponsavel]

    class temMultaCominatoria(ObjectProperty, FunctionalProperty):
        # Associação 1–0/1: uma obrigação pode ter uma multa cominatória
        domain = [Obrigacao]
        range  = [MultaCominatoria]

    class tipo(ObjectProperty, FunctionalProperty):
        # Liga Obrigacao -> TipoObrigacao (Fazer | NaoFazer)
        domain = [Obrigacao]
        range  = [TipoObrigacao]

    class solidariosMultaCominatoria(ObjectProperty):
        # Lista de responsáveis solidários da multa cominatória
        domain = [MultaCominatoria]
        range  = [PessoaResponsavel]

    # ======== PROPRIEDADES DADO (funcionais onde faz sentido) ========
    class descricao_obrigacao(DataProperty, FunctionalProperty):
        domain = [Obrigacao];  range = [str]

    class prazo_texto(DataProperty, FunctionalProperty):
        # Ex.: "90 dias" (texto extraído do acórdão)
        domain = [Obrigacao];  range = [str]

    class data_cumprimento(DataProperty, FunctionalProperty):
        # Use "YYYY-MM-DD" (string) para manter compat compatível com xsd:date
        domain = [Obrigacao];  range = [str]

    class texto_acordao(DataProperty):
        # Armazena o trecho relevante ou o acórdão inteiro, se necessário
        domain = [Acordao];    range = [str]

    # --- Multa cominatória (campos do prompt) ---
    class tem_multa_cominatoria(DataProperty, FunctionalProperty):
        # espelho booleano; opcional (pode ser inferido por temMultaCominatoria)
        domain = [Obrigacao];  range = [bool]

    class valor_multa_cominatoria(DataProperty, FunctionalProperty):
        # valor numérico (use float; se quiser precisão decimal, trate na camada de aplicação)
        domain = [MultaCominatoria];  range = [float]

    class periodo_multa_cominatoria(DataProperty, FunctionalProperty):
        # Ex.: "por dia", "por semana", "R$500/dia", etc. (texto do acórdão)
        domain = [MultaCominatoria];  range = [str]

    class e_multa_cominatoria_solidaria(DataProperty, FunctionalProperty):
        domain = [MultaCominatoria];  range = [bool]

    class nome_responsavel_multa_cominatoria(DataProperty):
        # redundante se modelar a pessoa como indivíduo; mantenho pq está no prompt
        domain = [MultaCominatoria];  range = [str]

    class documento_responsavel_multa_cominatoria(DataProperty):
        domain = [MultaCominatoria];  range = [str]

    # --- Metadados úteis das entidades-agente ---
    class nome(DataProperty, FunctionalProperty):
        domain = [PessoaResponsavel, OrgaoResponsavel]; range = [str]

    class documento(DataProperty, FunctionalProperty):
        # CPF/CNPJ; use máscara na aplicação
        domain = [PessoaResponsavel, OrgaoResponsavel]; range = [str]

# Salvar para inspecionar no Protégé
world.save(file="tce_model.owl", format="rdfxml")
print("Salvo em tce_model.owl")

Salvo em tce_model.owl


In [33]:
dul.SocialObject

DUL.SocialObject

In [34]:
dul

get_ontology("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")